# Investment Report
#### Done
* Use configuration file for confidential data (E-Mail, Google Sheet, etc.) **DONE**
* Write for-loop in E-Mail body for measures of single cryptocurrencies **DONE**
* Format decimals in pie-chart **DONE**
* Make currency and measure (Close, Max, Min, Open) flexible **DONE**
* Sort list 'Asset' alphabeticially **DONE**
* Write README and define format of Google Sheet input, G-Mail set-up & file paths **DONE**
* Make start date flexible (read from Google sheet, start one month prior on 1st) **DONE**
* Create second pie chart showing the allocation of purchase price **DONE**
* Create a table showing details by asset & formatted **DONE**
* Text details eliminated **DONE**
* Change E-Mail to 'more-secure apps' **DONE**

#### To Do
* Set up data import for stocks

In [26]:
print('Setting up the Environment')
#Setting up my environment
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime as dt

import plotly.express as px
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import gspread as gs

import kaleido

import yagmail

import configparser

import tempfile

Setting up the Environment


In [27]:
print('Importing Configurations File')
#Configuration File
config = configparser.ConfigParser()

config.read('Investment_Report.ini')

Importing Configurations File


['Investment_Report.ini']

In [28]:
#DEFINITIONS
#Google Sheet
API_filename = config['Google_Sheet']['API_filename']
gs_url = config['Google_Sheet']['gs_url']
worksheet = config['Google_Sheet']['worksheet']

#Historical Data
currency = config['Price_Data']['currency']
metric = config['Price_Data']['metric']

#Plot/Attachment paths
total_holdings_image_path = tempfile.NamedTemporaryFile(prefix='Total_Holdings_',suffix = '.png').name
holdings_by_type_image_path = tempfile.NamedTemporaryFile(prefix='Holdings_by_Asset_',suffix = '.png').name
allocation_of_holdings_image_path = tempfile.NamedTemporaryFile(prefix='Allocation_of_Holdings_',suffix = '.png').name

#E-Mail
send_to = config['E_Mail']['send_to']
send_from = config['E_Mail']['send_from']
app_password = config['E_Mail']['app_password']

In [29]:
print('Importing Data')
#DATA IMPORT
#GOOGLE SHEET TRANSACTIONS
gs1 = gs.service_account(API_filename)
gs2 = gs1.open_by_url(gs_url)
gs3 = gs2.worksheet(worksheet)
gs4 = gs3.get_all_records()

trans_data = pd.DataFrame(gs4)

#Formatting of date column
trans_data['Date'] = pd.to_datetime(trans_data['Date'])
trans_data = trans_data[['Date','Asset','Quantity']]

#DAILY CLOSE PRICES
#Set parameters
if trans_data['Date'].min().month == 1:
    start_month = 12
    start_year = (trans_data['Date'].min().year)-1
else:
    start_month = (trans_data['Date'].min().month)-1
    start_year = trans_data['Date'].min().year

start = dt.datetime(start_year,start_month,1)
end = dt.datetime.now()
crypto = sorted(trans_data['Asset'].unique().tolist())
colnames = []
first = True

#Import historical data
for ticker in crypto:
    try:
        data = web.DataReader(f'{ticker}-{currency}', 'yahoo', start, end)
        if first:
            combined = data[[metric]].copy()
            colnames.append(ticker)
            combined.columns = colnames
            first = False
        else:
            combined = combined.join(data[metric])
            colnames.append(ticker)
            combined.columns = colnames
    except:
        print (f'could not load Data for {ticker}')

#Re-shape data frame
combined.reset_index(inplace=True)
hist_data = pd.melt(combined, id_vars='Date', var_name='Asset',value_name=metric)

Importing Data


In [30]:
print('Manipulating Data')
#DATA MANIPULATION
#JOIN DATA
comb_data = pd.merge(trans_data,hist_data,how='right',on=['Date','Asset'])

#Calculation of accumulative, continuous sum for each crypto type, merge into one column and calculate the continuous value
for ticker in crypto:
    comb_data[f'Acc_Quantity_{ticker}'] = comb_data[(comb_data['Asset'] == f'{ticker}')]['Quantity'].cumsum().ffill()
    comb_data[f'Acc_Quantity_{ticker}'] = comb_data[f'Acc_Quantity_{ticker}'].fillna(value=0)

comb_data['Acc_Quantity'] = 0

for ticker in crypto:
    comb_data['Acc_Quantity'] += comb_data[f'Acc_Quantity_{ticker}']

comb_data['Pur_Value'] = comb_data['Quantity'] * comb_data[metric]
comb_data['Pur_Value'].fillna(value=0,inplace=True)

for ticker in crypto:
    comb_data[f'Acc_Pur_Value_{ticker}'] = comb_data[(comb_data['Asset'] == f'{ticker}')]['Pur_Value'].cumsum().ffill()
    comb_data[f'Acc_Pur_Value_{ticker}'] = comb_data[f'Acc_Pur_Value_{ticker}'].fillna(value=0)

comb_data['Acc_Pur_Value'] = 0

for ticker in crypto:
    comb_data['Acc_Pur_Value'] += comb_data[f'Acc_Pur_Value_{ticker}']

comb_data['Value'] = comb_data['Acc_Quantity'] * comb_data[metric]

comb_data['Pur_Value'] = comb_data['Quantity'] * comb_data[metric]
comb_data['Pur_Value'].fillna(value=0,inplace=True)

comb_data = comb_data[['Date','Asset','Quantity',metric,'Acc_Quantity','Value','Pur_Value','Acc_Pur_Value']]

#Total holdings
total_holdings = comb_data[['Date','Value','Pur_Value']].groupby('Date').sum()
total_holdings['Pur_Value'] = total_holdings['Pur_Value'].cumsum().ffill()

Manipulating Data


In [31]:
print('Analyzing Data')
#DATA ANALYSIS
#Total holdings measures
current_value = total_holdings.sort_values('Date',ascending=False).head(8)['Value']
max_value = round(total_holdings['Value'].max(), 2)
max_value_date = total_holdings['Value'].idxmax().strftime('%B %#d, %Y')
purchase_value = total_holdings.sort_values('Date',ascending=False).head(1)['Pur_Value']

purchase_value_only = round(purchase_value[0],2)
current_value_only = round(current_value[0],2)
past_value_only = round(current_value[1],2)
prior_week_value_only = round(current_value[7],2)

change_prior_day = round((current_value_only/past_value_only*100)-100,2)
change_prior_week = round((current_value_only/prior_week_value_only*100)-100,2)
change_purchase = round((current_value_only/purchase_value_only*100)-100,2)

total_performance = round((current_value_only-purchase_value_only),2)

#Measures by cryptocurrency
#All-time high
max_asset_value = dict.fromkeys(crypto, 0)
#All-time high date
max_asset_date = dict.fromkeys(crypto, 0)
#Current
current_asset_value = dict.fromkeys(crypto, 0)
#Prior week
prior_week_asset_value = dict.fromkeys(crypto, 0)
#Current purchase value
current_purchase_value = dict.fromkeys(crypto, 0)
#Quantity
quantity = dict.fromkeys(crypto, 0)
#Current Price
current_price = dict.fromkeys(crypto, 0)
for ticker in crypto:
    asset_data = comb_data[(comb_data['Asset'] == f'{ticker}')]
    if not asset_data.empty:
        asset_value = max(asset_data['Value'], default=0)
        max_asset_value[ticker] = asset_value
        asset_value = asset_data.set_index('Date')['Value'].idxmax()
        max_asset_date[ticker] = asset_value.strftime('%B %#d, %Y')
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Value']
        current_asset_value[ticker] = asset_value.iloc[0]
        asset_value = asset_data.sort_values('Date', ascending=False).head(8)['Value']
        prior_week_asset_value[ticker] = asset_value.iloc[7]
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Acc_Pur_Value']
        current_purchase_value[ticker] = asset_value.iloc[0]
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Acc_Quantity']
        quantity[ticker] = asset_value.iloc[0]
        asset_value = asset_data.sort_values('Date', ascending=False).head(1)['Close']
        current_price[ticker] = asset_value.iloc[0]

#Overview Table
overview_table = pd.DataFrame({'Quantity':pd.Series(quantity),
                                'Current Price':pd.Series(current_price),
                                'Current Value':pd.Series(current_asset_value),
                                'Purchase Value':pd.Series(current_purchase_value),
                                'All-time High':pd.Series(max_asset_value),
                                'All-time High Date':pd.Series(max_asset_date),
                                'Prior Week Value':pd.Series(prior_week_asset_value)})

overview_table['Gain/Loss'] = overview_table['Current Value'] - overview_table['Purchase Value']
overview_table['Gain/Loss %'] = (overview_table['Current Value'] / overview_table['Purchase Value']*100)-100
overview_table['Gain/Loss to Prior Week'] = overview_table['Current Value'] - overview_table['Prior Week Value']
overview_table['Gain/Loss to Prior Week %'] = (overview_table['Current Value'] / overview_table['Prior Week Value']*100)-100

overview_table = overview_table[['Quantity','Current Price','Current Value','Purchase Value','Gain/Loss','Gain/Loss %','Prior Week Value','Gain/Loss to Prior Week','Gain/Loss to Prior Week %','All-time High','All-time High Date']]

overview_table = overview_table.round(2)

Analyzing Data


In [32]:
print('Creating Visualizations')
#DATA VISUALIZATION
#Development total holdings
fig = px.line(total_holdings,x=total_holdings.index,y=['Value','Pur_Value'],width=2000,height=1000)
fig.update_layout(title_text='Development Total Holdings', title_x=0.5)
fig.update_yaxes(title=f'Value in {currency}')
fig.write_image(total_holdings_image_path)

#Development by Type
fig_2 = px.line(comb_data,x='Date',y='Value', color='Asset',width=2000,height=1000)
fig_2.update_layout(title_text='Development Holdings by Type', title_x=0.5)
fig_2.update_yaxes(title=f'Value in {currency}')
fig_2.write_image(holdings_by_type_image_path)

#Allocation of Holdings
distinct_values = comb_data['Asset'].nunique()

fig_3 = make_subplots(rows=1, cols=2 ,specs=[[{"type": "pie"}, {"type": "pie"}]])

fig_3.add_trace(go.Pie(
     values=round(comb_data.sort_values('Date', ascending=False).head(distinct_values).sort_index()['Value'],2),
     labels=crypto,
     domain=dict(x=[0, 0.5]),
     name="Current Value"), 
     row=1, col=1)

fig_3.add_trace(go.Pie(
     values=round(comb_data.sort_values('Date', ascending=False).head(distinct_values).sort_index()['Acc_Pur_Value'],2),
     labels=crypto,
     domain=dict(x=[0, 0.5]),
     name="Purchase Value"), 
     row=1, col=2)

fig_3.update_traces(
                hole=.5, 
                hoverinfo="label+percent+value",
                textposition='outside',
                texttemplate='<b>%{label} </b><br>%{value} <br>%{percent:.2%}')


fig_3.update_layout(
        title_text=f'Allocation of Holdings in {currency} (Current vs. Purchase Value)', 
        title_x=0.5,
        annotations=[dict(text=f'<b>{current_value_only}', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text=f'<b>{purchase_value_only}', x=0.81, y=0.5, font_size=20, showarrow=False)], 
        showlegend=False,
        autosize=False,
        width=1500,
        height=800,)

fig_3.write_image(allocation_of_holdings_image_path)

Creating Visualizations


In [33]:
print('Preparing E-Mail')
#E-MAIL NOTIFICATION

subject = 'Investment Report'
body =  f'Current total holdings: {current_value_only}{currency} ({change_prior_week}% to prior week)\n\
   - Gain/Loss of {total_performance}{currency} ({change_purchase}%) to purchase value, which was {purchase_value_only}{currency}\n\
   - All-time high of {max_value}{currency} on {max_value_date}\n\n'

content = [body, overview_table, total_holdings_image_path, holdings_by_type_image_path, allocation_of_holdings_image_path]

with yagmail.SMTP(send_from, app_password) as yag:
    yag.send(send_to, subject, content)

print('E-Mail Sent')

# Measure by asset in text format
# for ticker in crypto:
#     latest = round(current_asset_value[ticker],2)
#     if latest == 0:
#         continue
#     prior = round(prior_week_asset_value[ticker],2)
#     change_prior_week = round((latest/prior*100)-100,2)
#     purchase = round(current_purchase_value[ticker],2)
#     performance = round((latest - purchase),2)
#     change_to_purchase = round((latest/purchase*100)-100,2)
#     all_time_high = round(max_asset_value[ticker],2)
#     date = max_asset_date[ticker]

#     body_2 = f'     t/o {ticker}: {latest}{currency} ({change_prior_week}% to prior week)\n\
#        - Gain/Loss of {performance}{currency} ({change_to_purchase}%) to purchase value, which was {purchase}{currency}\n\
#        - All-time high of {all_time_high}{currency} on {date}\n\n'

Preparing E-Mail
E-Mail Sent
